In [ ]:
import torch
from google.colab import drive
import torch.optim as optim
import torch.nn as nn
from sklearn import preprocessing
import pandas as pd
from google.colab import drive
import random as rd
import numpy as np

drive.mount('/content/drive')
apple_csv = "/content/drive/My Drive/Colab Notebooks/AAPL.csv"


aapl_df = pd.read_csv(apple_csv)
print(aapl_df.tail())
# A random starting date
starting_date = rd.randint(0,3000)
x_data = []
y_data = [] 
# The goal: to predict wether a stock will increase or decrease on a given day
# Our time step is 30 days, meaning we use the past 30 days to predict the next day
# We will be predicting on and for 1 feature, the closing price of the stock
# The x_dataset will be time_step * num_predictions
# The y_dataset will be num_features * num_predictions
# We will be doing predictions for 100 days this tutorial
# The following algorithm creates the raw data sets, and converts the y values to binary values
for x in range(3100):
  date = starting_date + x
  if((x+620)%31 == 0):
    if(aapl_df.loc[date,"Close"]>aapl_df.loc[date-1,"Close"]):
      # The 1 represents a buy signal
      y_data.append(1)
    else:
      # The 0 represents a sell signal
      y_data.append(0)
  else:
    x_data.append(aapl_df.loc[date,"Close"])
print(len(x_data))
#Normalizing the data
normal_x = preprocessing.scale(x_data) 

#Converting the data to tensors
x_tensor = torch.FloatTensor(normal_x).float()
y_tensor = torch.FloatTensor(y_data).float()

# Splitting the data into training and test(80% training, 20% test)
x_val_size = x_tensor.shape[0] // 5
y_val_size = y_tensor.shape[0] // 5
x_train = x_tensor[x_val_size:].cuda()
y_train = y_tensor[y_val_size:].cuda()
x_valid = x_tensor[:x_val_size].cuda()
y_valid = y_tensor[:y_val_size].cuda()

# Splitting the input vector into batches for training and test data sets per timestep
x_train = torch.split(x_train,30)
x_valid = torch.split(x_valid,30)

# The deep network
class LogisticRegressor(torch.nn.Module):
  def __init__(self):
        super(LogisticRegressor, self).__init__()
        layers = []
        num_hidden_layers = 2
        input_dim = 30
        for i in range(num_hidden_layers):
            layers.append(nn.Linear(30,30))
            layers.append(nn.ReLU())
        self.seq = nn.Sequential(*layers, nn.Linear(30, 1), nn.Sigmoid())        
  def forward(self, x):
        return self.seq(x)

# Hyperparameter Optimization
model = LogisticRegressor().cuda()
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())
epochs = 100

acc = 0

#Training loop
def train(x,y,iteration, epoch):
        global acc
        model.train()
        optimizer.zero_grad()
        y_train_pred = model(x)
        loss = criterion(y_train_pred,y)
        if(y == torch.round(y_train_pred)):
          acc = acc+1
        if(iteration == 79):
            if(epoch % 5 == 0):
                print("Accuracy: {:.4%}".format(acc/80))
            acc = 0
        loss.backward()
        optimizer.step()

#Training
for y in range(epochs):
  for z in range(80):
    train(x_train[z],y_train[z],z, y)

# validation and evaluation
val_acc = 0
model.eval()
for x in range(20):
  global val_acc
  y_pred = model(x_valid[x])
  if(torch.round(y_pred) == y_valid[x]):
      val_acc = val_acc + 1
print(val_acc/20)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
            Date        Open        High  ...       Close   Adj Close      Volume
9858  2020-01-17  316.269989  318.739990  ...  318.730011  318.730011  34454100.0
9859  2020-01-21  317.190002  319.019989  ...  316.570007  316.570007  27710800.0
9860  2020-01-22  318.579987  319.989990  ...  317.700012  317.700012  25458100.0
9861  2020-01-23  317.920013  319.559998  ...  319.230011  319.230011  26118000.0
9862  2020-01-24  320.250000  323.329987  ...  318.309998  318.309998  36600500.0

[5 rows x 7 columns]
3000
<generator object Module.parameters at 0x7fece4553c50>


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Accuracy: 41.2500%
Accuracy: 56.2500%
Accuracy: 57.5000%
Accuracy: 57.5000%
Accuracy: 57.5000%
Accuracy: 60.0000%
Accuracy: 60.0000%
Accuracy: 56.2500%
Accuracy: 60.0000%
Accuracy: 61.2500%
Accuracy: 61.2500%
Accuracy: 65.0000%
Accuracy: 67.5000%
Accuracy: 68.7500%
Accuracy: 73.7500%
Accuracy: 71.2500%
Accuracy: 77.5000%
Accuracy: 76.2500%
Accuracy: 78.7500%
Accuracy: 77.5000%
0.35


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
